In [33]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import requests
import json
train_data=pd.read_csv("Finetune_data.csv")


In [34]:
train_data.head()

,Indicators (Select from drop-down list),Description Simplified,Geographic Coverage,Format,Spatial Resolution,Temporal Resolution,Temporal Extent,Latency,Source/Link,Project,Data Visualization
0,Disasters,NASA's Black Marble night lights data provide ...,"Puerto Rico, New Orleans","tif, jpeg, mpeg",NaN,NaN,2017,NaN,https://svs.gsfc.nasa.gov/4658#:~:text=NASA%27...,Disaster Dashboard,True
1,Urban Flooding,The Flood Dashboard brings together mutliple N...,Global,Interactive Visualization,Varies,Varies,Varies,1 Day,https://maps.disasters.nasa.gov/arcgis/apps/op...,NaN,True
2,Human Dimensions,The Global Cyclone Proportional Economic Loss ...,Global,"ASCII, PDF, PNG, WMS",0.0417 degrees,NaN,2000-01-01 to 2000-12-31,NaN,https://doi.org/10.7927/H44F1NNF,NDH - Natural Disaster Hotspots,True
3,Food Availability,The Global Food Security-support Analysis Data...,Africa,GeoTIFF,30m,NaN,2013-01-01 to 2016-06-30,NaN,https://doi.org/10.5067/MEaSUREs/GFSAD/GFSAD30...,MEaSUREs - Making Earth System Data Records fo...,False
4,Climate Change,The MuSLI Multi-Source Land Surface Phenology ...,North America,netCDF-4,30m,NaN,2016-01-01 to 2019-12-31,NaN,https://doi.org/10.5067/Community/MuSLI/MSLSP3...,NaN,False


In [36]:
final_data={}
for each_data in train_data.iterrows():
    final_data[each_data[1][8]]={"Indicators":each_data[1][0],
              "Description":each_data[1][1],
              "Geographic_Coverage":each_data[1][2],
              "Format":each_data[1][3],
              "Spatial_Resolution":each_data[1][4],
              "Temporal_Resolution":each_data[1][5],
              "Temporal_Extent":each_data[1][6],
              "Latency":each_data[1][7],
                "Project":each_data[1][9],
                "Data_Visualization":each_data[1][10]}
    

In [37]:
for url,value in final_data.items():
    response=requests.get(url)
    html_page = response.text
    soup = BeautifulSoup(html_page, "html.parser")
    text=soup.get_text()
    text = re.sub(r'[\t\n\r\f\v]+', '', text)
    text=re.sub(r'[^\w\s]', '', text)
    final_data[url]["text"]=text


In [41]:
final_data

{'https://svs.gsfc.nasa.gov/4658#:~:text=NASA%27s%20Black%20Marble%20night%20lights%20used%20to%20examine%20disaster%20recovery%20in%20Puerto%20Rico,-Visualizations%20by%20Kel&text=At%20night%2C%20Earth%20is%20lit,Puerto%20Rico%27s%20lights%20went%20out.': {'Indicators': 'Disasters',
  'Description': "NASA's Black Marble night lights data provide a neighborhood-scale map of energy use in communities in Puerto Rico and New Orleans as the electricity grid was slowly restored after Hurricane Maria and Hurricane Ida. These data can then be used to study the relationship between restoration rates and the remoteness or other aspects of communities across these areas. These data are a product of the NASA/NOAA (National Oceanic and Atmospheric Administration) Suomi National Polar-orbiting Partnership satellite, using USGS (United States Geologic Survey)-NASA Landsat data, as well as Google's OpenStreetMap data.",
  'Geographic_Coverage': 'Puerto Rico, New Orleans',
  'Format': 'tif, jpeg, mpeg

In [42]:
_SYSTEM_PROMPT = (
        "Extract metadata and entities details accurately from my requests." +
        "Metadata extraction for Indicators should clearly be one of the element in the list [“Disasters”,“Human Dimensions”,“Food Availability”,“Health & Air Quality”,“Water Availability”, “Extreme Heat”,“Urban Flooding”,“Climate]. Do not create new element for indicators "
        "Make sure the following fields follow the following regex pattern"+
        "Spatial_Resolution: ^(\d+(\.\d+)? [a-zA-Z]+|varies|N/A)$"+
        "Temporal_Resolution: ^(\d+(\.\d+)? [a-zA-Z]+|N/A|varies|weekly|monthly|daily|yearly|varies-multiple datasets included|Daily < Weekly|Hourly < Daily|Weekly < Monthly|Monthly < Yearly|1 minute)$"+
        "Temporal_Extent: ^(\d{4}-\d{2}-\d{2} to present|present|\d{4}-\d{2}-\d{2} (?:to|until) present|\d{4}-\d{2}-\d{2} to \d{4}-\d{2}-\d{2}|varies(?:- multiple datasets (?:included|available))?)$"
    )

In [47]:
train_data=[]
for url,value in final_data.items():
    data={"messages": [{"role": "system", "content":_SYSTEM_PROMPT},{"role": "user", "content": f'{value["text"]}'}, {"role": "assistant", "content": {"Indicators":value["Indicators"],"Description":value["Description"],"Geographic_Coverage":value["Geographic_Coverage"], "Format":value["Format"],"Spatial_Resolution":value["Spatial_Resolution"],"Temporal_Resolution":value["Temporal_Resolution"],"Temporal_Extent":value["Temporal_Extent"],"Latency":value["Latency"],"Project":value["Project"],"Data_Visualization":value["Data_Visualization"]}}]}
    train_data.append(data)
    
with open("train_data.jsonl", 'w') as jsonl_file:
    for record in train_data:
        json.dump(record, jsonl_file)  # Write JSON object
        jsonl_file.write('\n')  # Add newline character to separate objects




In [48]:
print(train_data[0])

{'messages': [{'role': 'system', 'content': 'Extract metadata and entities details accurately from my requests.Metadata extraction for Indicators should clearly be one of the element in the list [“Disasters”,“Human Dimensions”,“Food Availability”,“Health & Air Quality”,“Water Availability”, “Extreme Heat”,“Urban Flooding”,“Climate]. Do not create new element for indicators Make sure the following fields follow the following regex patternSpatial_Resolution: ^(\\d+(\\.\\d+)? [a-zA-Z]+|varies|N/A)$Temporal_Resolution: ^(\\d+(\\.\\d+)? [a-zA-Z]+|N/A|varies|weekly|monthly|daily|yearly|varies-multiple datasets included|Daily < Weekly|Hourly < Daily|Weekly < Monthly|Monthly < Yearly|1 minute)$Temporal_Extent: ^(\\d{4}-\\d{2}-\\d{2} to present|present|\\d{4}-\\d{2}-\\d{2} (?:to|until) present|\\d{4}-\\d{2}-\\d{2} to \\d{4}-\\d{2}-\\d{2}|varies(?:- multiple datasets (?:included|available))?)$'}, {'role': 'user', 'content': 'NASA SVS  NASAs Black Marble night lights used to examine disaster reco